In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import tensorflow as tf
import tensorflow.keras.layers as tfl
from tensorflow.keras.layers.experimental.preprocessing import RandomFlip, RandomRotation, RandomZoom
import io
import imageio
from tensorflow.keras.preprocessing import image_dataset_from_directory
import seaborn as sns

from sklearn.model_selection import train_test_split


import PIL
from PIL import ImageFont, ImageDraw, Image

from ydata_profiling import ProfileReport

In [ ]:
dir_train = 'A://workspace//datasets//tiny_imagenet//train.parquet'
dir_valid = 'A://workspace//datasets//tiny_imagenet//valid.parquet'

In [ ]:
train_dataset = pd.read_parquet(dir_train, engine='pyarrow')
valid_dataset = pd.read_parquet(dir_valid, engine='pyarrow')

In [ ]:
train_dataset['image_matrix'] = train_dataset['image'].apply(lambda x: np.array(Image.open(io.BytesIO(x['bytes']))))

In [ ]:
plt.figure(figsize=(10, 10))
for i, image in train_dataset['image_matrix'][:4500:500].items():
    ax = plt.subplot(3, 3, i//500 + 1)
    plt.imshow(image)
    plt.axis("off")

In [ ]:
len(train_dataset['image_matrix'].iloc[1])

In [ ]:
# check count and amount of classes
labels_count = list(train_dataset['label'].value_counts())
len(labels_count), set(labels_count)

In [ ]:
# train_dataset with stratify
img_train_orig, img_test_orig, label_train, label_test = train_test_split(train_dataset['image_matrix'], train_dataset['label'], test_size=0.2, random_state=42,
                                                               stratify=train_dataset['label'])

In [ ]:
img_train_orig.shape, label_train.shape, img_test_orig.shape, label_test.shape

In [ ]:
def data_augmenter():
    data_augmentation = tf.keras.Sequential()
    data_augmentation.add(RandomFlip("horizontal"))
    data_augmentation.add(RandomFlip("vertical"))
    data_augmentation.add(RandomRotation(0.2))
    data_augmentation.add(RandomZoom(height_factor=(-0.2, 0.2), width_factor=(-0.2, 0.2)))
    
    return data_augmentation

In [ ]:
data_augmentation = data_augmenter()

plt.figure(figsize=(10, 10))
first_image = img_train_orig.iloc[3]
for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    augmented_image = data_augmentation(tf.expand_dims(first_image, 0))
    plt.imshow(augmented_image[0] / 255)
    plt.axis('off')

In [ ]:
img_train = img_train_orig/255
img_test = img_test_orig/255

img_train = (np.repeat(im[:, :, np.newaxis], 3, axis=2) if im.shape == (64,64) else im for im in img_train)
img_test = (np.repeat(im[:, :, np.newaxis], 3, axis=2) if im.shape == (64,64) else im for im in img_test)

img_train = np.stack(list(img_train))
img_test = np.stack(list(img_test))

img_train.shape

In [ ]:
label_train = pd.get_dummies(label_train, dtype='float32')
label_test= pd.get_dummies(label_test, dtype='float32')

with tf.device('/cpu:0'):
    label_train = tf.convert_to_tensor(label_train.values, np.float32)
    label_test = tf.convert_to_tensor(label_test.values, np.float32)
    img_train = tf.convert_to_tensor(img_train, np.float32)
    img_test = tf.convert_to_tensor(img_test, np.float32)

train_dataset = tf.data.Dataset.from_tensor_slices((img_train, label_train)).batch(64)
test_dataset = tf.data.Dataset.from_tensor_slices((img_test, label_test)).batch(64)

# Написать блоки кода, чтобы можно было через цикл собирать модели и тестить. 

# Test

In [ ]:

def convolutional_model(input_shape, drop_out=0.3):
    input_img = tf.keras.Input(shape=input_shape)
    X = data_augmentation(input_img) # это с каждой эпохой происходит ци как? это вообще здесь должно быть?

    X = tf.keras.layers.Conv2D(filters = 32 , kernel_size= (3,3), strides = (1,1), padding='same', activation='relu')(X)
    X = tfl.BatchNormalization()(X)
    X = tf.keras.layers.Conv2D(filters = 32 , kernel_size= (3,3), strides = (1,1), padding='same', activation='relu')(X)
    X = tfl.BatchNormalization()(X)
    X = tf.keras.layers.Conv2D(filters = 32 , kernel_size= (3,3), strides = (1,1), padding='same', activation='relu')(X)
    X = tfl.BatchNormalization()(X)
    
    X = tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(2, 2), padding='valid')(X)
    X = tf.keras.layers.Dropout(drop_out)(X)

    X = tf.keras.layers.Conv2D(filters = 64 , kernel_size= (3,3), strides = (1,1), padding='same', activation='relu')(X)
    X = tfl.BatchNormalization()(X)
    X = tf.keras.layers.Conv2D(filters = 64 , kernel_size= (3,3), strides = (1,1), padding='same', activation='relu')(X)
    X = tfl.BatchNormalization()(X)
    X = tf.keras.layers.Conv2D(filters = 64 , kernel_size= (3,3), strides = (1,1), padding='same', activation='relu')(X)
    X = tfl.BatchNormalization()(X)
    
    X = tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(2, 2), padding='valid')(X)
    X = tf.keras.layers.Dropout(drop_out)(X)

    X = tf.keras.layers.Conv2D(filters = 128 , kernel_size= (3,3), strides = (1,1), padding='same', activation='relu')(X)
    X = tfl.BatchNormalization()(X)
    X = tf.keras.layers.Conv2D(filters = 128 , kernel_size= (3,3), strides = (1,1), padding='same', activation='relu')(X)
    X = tfl.BatchNormalization()(X)
    X = tf.keras.layers.Conv2D(filters = 128 , kernel_size= (3,3), strides = (1,1), padding='same', activation='relu')(X)
    X = tfl.BatchNormalization()(X)
    
    X = tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(2, 2), padding='valid')(X)
    X = tf.keras.layers.Dropout(drop_out)(X)

    X = tf.keras.layers.Conv2D(filters = 256 , kernel_size= (3,3), strides = (1,1), padding='same', activation='relu')(X)
    X = tfl.BatchNormalization()(X)
    X = tf.keras.layers.Conv2D(filters = 256 , kernel_size= (3,3), strides = (1,1), padding='same', activation='relu')(X)
    X = tfl.BatchNormalization()(X)
    X = tf.keras.layers.Conv2D(filters = 256 , kernel_size= (3,3), strides = (1,1), padding='same', activation='relu')(X)
    X = tfl.BatchNormalization()(X)
    
    X = tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(2, 2), padding='valid')(X)
    X = tf.keras.layers.Dropout(drop_out)(X)

    X = tf.keras.layers.Flatten()(X)

    X = tf.keras.layers.Dense(units=512, activation='relu')(X)
    X = tfl.BatchNormalization(axis = 1)(X)
    X = tf.keras.layers.Dropout(0.5)(X)

    X = tf.keras.layers.Dense(units=200, activation='softmax')(X)
    
    model = tf.keras.Model(inputs=input_img, outputs=X)
    return model

conv_model = convolutional_model((64, 64, 3))
print(conv_model.summary())

conv_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

history = conv_model.fit(train_dataset, validation_data=test_dataset, batch_size=1024, epochs=100,workers=-1, callbacks=[tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        min_delta=0.01,
        patience=10,
        verbose=0,
        mode='auto',
        baseline=None,
        restore_best_weights=False
    )])

# Лучшие модели
## Порядок слоев
- Conv
- Batch
- Relu?
- MaxPool
- DropOut

In [ ]:
# 40 20
def convolutional_model(input_shape, drop_out=0.3):
    input_img = tf.keras.Input(shape=input_shape)
    X = data_augmentation(input_img) # это с каждой эпохой происходит ци как? это вообще здесь должно быть?

    X = tf.keras.layers.Conv2D(filters = 32 , kernel_size= (3,3), strides = (1,1), padding='same', activation='relu')(X)
    X = tfl.BatchNormalization(axis=3)(X)
    X = tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(2, 2), padding='valid')(X)
    X = tf.keras.layers.Dropout(drop_out)(X)

    X = tf.keras.layers.Conv2D(filters = 64 , kernel_size= (3,3), strides = (1,1), padding='same', activation='relu')(X)
    X = tfl.BatchNormalization(axis=3)(X)
    X = tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(2, 2), padding='valid')(X)
    X = tf.keras.layers.Dropout(drop_out)(X)

    X = tf.keras.layers.Conv2D(filters = 128 , kernel_size= (3,3), strides = (1,1), padding='same', activation='relu')(X)
    X = tfl.BatchNormalization(axis=3)(X)
    X = tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(2, 2), padding='valid')(X)
    X = tf.keras.layers.Dropout(drop_out)(X)

    X = tf.keras.layers.Flatten()(X)
    X = tf.keras.layers.Dense(units=2048, activation='relu')(X)
    X = tf.keras.layers.Dense(units=1024, activation='relu')(X)
    X = tfl.BatchNormalization(axis = 1)(X)
    X = tf.keras.layers.Dropout(0.5)(X)
    X = tf.keras.layers.Dense(units=512, activation='relu')(X)

    X = tf.keras.layers.Dense(units=200, activation='softmax')(X)
    
    model = tf.keras.Model(inputs=input_img, outputs=X)
    return model

conv_model = convolutional_model((64, 64, 3))
print(conv_model.summary())

conv_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

history = conv_model.fit(train_dataset, validation_data=test_dataset, batch_size=16, epochs=100,workers=-1, callbacks=[tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        min_delta=0.01,
        patience=10,
        verbose=0,
        mode='auto',
        baseline=None,
        restore_best_weights=False
    )])

In [ ]:

def convolutional_model(input_shape, drop_out=0.3):
    input_img = tf.keras.Input(shape=input_shape)
    X = data_augmentation(input_img) # это с каждой эпохой происходит ци как? это вообще здесь должно быть?

    X = tf.keras.layers.Conv2D(filters = 32 , kernel_size= (3,3), strides = (1,1), padding='same', activation='relu')(X)
    X = tfl.BatchNormalization()(X)
    X = tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(2, 2), padding='valid')(X)
    X = tf.keras.layers.Dropout(drop_out)(X)

    X = tf.keras.layers.Conv2D(filters = 64 , kernel_size= (3,3), strides = (1,1), padding='same', activation='relu')(X)
    X = tfl.BatchNormalization()(X)
    X = tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(2, 2), padding='valid')(X)
    X = tf.keras.layers.Dropout(drop_out)(X)

    X = tf.keras.layers.Conv2D(filters = 128 , kernel_size= (3,3), strides = (1,1), padding='same', activation='relu')(X)
    X = tfl.BatchNormalization()(X)
    X = tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(2, 2), padding='valid')(X)
    X = tf.keras.layers.Dropout(drop_out)(X)

    X = tf.keras.layers.Conv2D(filters = 256 , kernel_size= (3,3), strides = (1,1), padding='same', activation='relu')(X)
    X = tfl.BatchNormalization()(X)
    X = tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(2, 2), padding='valid')(X)
    X = tf.keras.layers.Dropout(drop_out)(X)

    X = tf.keras.layers.Flatten()(X)
    
    X = tf.keras.layers.Dense(units=512, activation='relu')(X)
    X = tfl.BatchNormalization(axis = 1)(X)
    X = tf.keras.layers.Dropout(0.5)(X)
    # X = tf.keras.layers.Dense(units=512, activation='relu')(X)

    X = tf.keras.layers.Dense(units=200, activation='softmax')(X)
    
    model = tf.keras.Model(inputs=input_img, outputs=X)
    return model

conv_model = convolutional_model((64, 64, 3))
print(conv_model.summary())

conv_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

history = conv_model.fit(train_dataset, validation_data=test_dataset, batch_size=1024, epochs=100,workers=-1, callbacks=[tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        min_delta=0.01,
        patience=10,
        verbose=0,
        mode='auto',
        baseline=None,
        restore_best_weights=False
    )])

In [ ]:

def convolutional_model(input_shape, drop_out=0.3):
    input_img = tf.keras.Input(shape=input_shape)
    X = data_augmentation(input_img) # это с каждой эпохой происходит ци как? это вообще здесь должно быть?

    X = tf.keras.layers.Conv2D(filters = 32 , kernel_size= (3,3), strides = (1,1), padding='same', activation='relu')(X)
    X = tf.keras.layers.Conv2D(filters = 32 , kernel_size= (3,3), strides = (1,1), padding='same', activation='relu')(X)
    X = tf.keras.layers.Conv2D(filters = 32 , kernel_size= (3,3), strides = (1,1), padding='same', activation='relu')(X)
    
    X = tfl.BatchNormalization()(X)
    X = tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(2, 2), padding='valid')(X)
    X = tf.keras.layers.Dropout(drop_out)(X)

    X = tf.keras.layers.Conv2D(filters = 64 , kernel_size= (3,3), strides = (1,1), padding='same', activation='relu')(X)
    X = tf.keras.layers.Conv2D(filters = 64 , kernel_size= (3,3), strides = (1,1), padding='same', activation='relu')(X)
    X = tf.keras.layers.Conv2D(filters = 64 , kernel_size= (3,3), strides = (1,1), padding='same', activation='relu')(X)
    
    X = tfl.BatchNormalization()(X)
    X = tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(2, 2), padding='valid')(X)
    X = tf.keras.layers.Dropout(drop_out)(X)

    X = tf.keras.layers.Conv2D(filters = 128 , kernel_size= (3,3), strides = (1,1), padding='same', activation='relu')(X)
    X = tf.keras.layers.Conv2D(filters = 128 , kernel_size= (3,3), strides = (1,1), padding='same', activation='relu')(X)
    X = tf.keras.layers.Conv2D(filters = 128 , kernel_size= (3,3), strides = (1,1), padding='same', activation='relu')(X)
    
    X = tfl.BatchNormalization()(X)
    X = tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(2, 2), padding='valid')(X)
    X = tf.keras.layers.Dropout(drop_out)(X)


    X = tf.keras.layers.Flatten()(X)


    X = tf.keras.layers.Dense(units=200, activation='softmax')(X)
    
    model = tf.keras.Model(inputs=input_img, outputs=X)
    return model

conv_model = convolutional_model((64, 64, 3))
print(conv_model.summary())

conv_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

history = conv_model.fit(train_dataset, validation_data=test_dataset, batch_size=1024, epochs=100,workers=-1, callbacks=[tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        min_delta=0.01,
        patience=10,
        verbose=0,
        mode='auto',
        baseline=None,
        restore_best_weights=False
    )])

In [ ]:
# 40 30
def convolutional_model(input_shape, drop_out=0.3):
    input_img = tf.keras.Input(shape=input_shape)
    X = data_augmentation(input_img) # это с каждой эпохой происходит ци как? это вообще здесь должно быть?

    X = tf.keras.layers.Conv2D(filters = 32 , kernel_size= (3,3), strides = (1,1), padding='same', activation='relu')(X)
    X = tf.keras.layers.Conv2D(filters = 32 , kernel_size= (3,3), strides = (1,1), padding='same', activation='relu')(X)
    X = tf.keras.layers.Conv2D(filters = 32 , kernel_size= (3,3), strides = (1,1), padding='same', activation='relu')(X)
    
    X = tfl.BatchNormalization()(X)
    X = tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(2, 2), padding='valid')(X)
    X = tf.keras.layers.Dropout(drop_out)(X)

    X = tf.keras.layers.Conv2D(filters = 64 , kernel_size= (3,3), strides = (1,1), padding='same', activation='relu')(X)
    X = tf.keras.layers.Conv2D(filters = 64 , kernel_size= (3,3), strides = (1,1), padding='same', activation='relu')(X)
    X = tf.keras.layers.Conv2D(filters = 64 , kernel_size= (3,3), strides = (1,1), padding='same', activation='relu')(X)
    
    X = tfl.BatchNormalization()(X)
    X = tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(2, 2), padding='valid')(X)
    X = tf.keras.layers.Dropout(drop_out)(X)

    X = tf.keras.layers.Conv2D(filters = 128 , kernel_size= (3,3), strides = (1,1), padding='same', activation='relu')(X)
    X = tf.keras.layers.Conv2D(filters = 128 , kernel_size= (3,3), strides = (1,1), padding='same', activation='relu')(X)
    X = tf.keras.layers.Conv2D(filters = 128 , kernel_size= (3,3), strides = (1,1), padding='same', activation='relu')(X)
    
    X = tfl.BatchNormalization()(X)
    X = tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(2, 2), padding='valid')(X)
    X = tf.keras.layers.Dropout(drop_out)(X)

    X = tf.keras.layers.Conv2D(filters = 256 , kernel_size= (3,3), strides = (1,1), padding='same', activation='relu')(X)
    X = tf.keras.layers.Conv2D(filters = 256 , kernel_size= (3,3), strides = (1,1), padding='same', activation='relu')(X)
    X = tf.keras.layers.Conv2D(filters = 256 , kernel_size= (3,3), strides = (1,1), padding='same', activation='relu')(X)
    
    X = tfl.BatchNormalization()(X)
    X = tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(2, 2), padding='valid')(X)
    X = tf.keras.layers.Dropout(drop_out)(X)

    X = tf.keras.layers.Flatten()(X)

    # X = tf.keras.layers.Dense(units=1024, activation='relu')(X)
    # X = tfl.BatchNormalization(axis = 1)(X)
    # X = tf.keras.layers.Dropout(0.5)(X)
    X = tf.keras.layers.Dense(units=512, activation='relu')(X)
    X = tfl.BatchNormalization(axis = 1)(X)
    X = tf.keras.layers.Dropout(0.5)(X)
    # X = tf.keras.layers.Dense(units=512, activation='relu')(X)

    X = tf.keras.layers.Dense(units=200, activation='softmax')(X)
    
    model = tf.keras.Model(inputs=input_img, outputs=X)
    return model

conv_model = convolutional_model((64, 64, 3))
print(conv_model.summary())

conv_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

history = conv_model.fit(train_dataset, validation_data=test_dataset, batch_size=1024, epochs=100,workers=-1, callbacks=[tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        min_delta=0.01,
        patience=10,
        verbose=0,
        mode='auto',
        baseline=None,
        restore_best_weights=False
    )])

In [ ]:
# 60 40
def convolutional_model(input_shape, drop_out=0.3):
    input_img = tf.keras.Input(shape=input_shape)
    X = data_augmentation(input_img)

    X = tf.keras.layers.Conv2D(filters = 32 , kernel_size= (3,3), strides = (1,1), padding='same', activation='relu')(X)
    X = tf.keras.layers.Conv2D(filters = 32 , kernel_size= (3,3), strides = (1,1), padding='same', activation='relu')(X)
    X = tf.keras.layers.Conv2D(filters = 32 , kernel_size= (3,3), strides = (1,1), padding='same', activation='relu')(X)
    
    X = tf.keras.layers.Conv2D(filters = 64 , kernel_size= (3,3), strides = (1,1), padding='same', activation='relu')(X)
    X = tf.keras.layers.Conv2D(filters = 64 , kernel_size= (3,3), strides = (1,1), padding='same', activation='relu')(X)
    X = tf.keras.layers.Conv2D(filters = 64 , kernel_size= (3,3), strides = (1,1), padding='same', activation='relu')(X)
    
    X = tf.keras.layers.Conv2D(filters = 128 , kernel_size= (3,3), strides = (1,1), padding='same', activation='relu')(X)
    X = tf.keras.layers.Conv2D(filters = 128 , kernel_size= (3,3), strides = (1,1), padding='same', activation='relu')(X)
    X = tf.keras.layers.Conv2D(filters = 128 , kernel_size= (3,3), strides = (1,1), padding='same', activation='relu')(X)
    
    X = tfl.BatchNormalization()(X)
    X = tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(2, 2), padding='valid')(X)
    X = tf.keras.layers.Dropout(drop_out)(X)

    X = tf.keras.layers.Conv2D(filters = 256 , kernel_size= (3,3), strides = (1,1), padding='same', activation='relu')(X)
    X = tf.keras.layers.Conv2D(filters = 256 , kernel_size= (3,3), strides = (1,1), padding='same', activation='relu')(X)
    X = tf.keras.layers.Conv2D(filters = 256 , kernel_size= (3,3), strides = (1,1), padding='same', activation='relu')(X)
    
    X = tfl.BatchNormalization()(X)
    X = tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(2, 2), padding='valid')(X)
    X = tf.keras.layers.Dropout(drop_out)(X)

    X = tf.keras.layers.Conv2D(filters = 512 , kernel_size= (3,3), strides = (1,1), padding='same', activation='relu')(X)
    X = tf.keras.layers.Conv2D(filters = 512 , kernel_size= (3,3), strides = (1,1), padding='same', activation='relu')(X)
    X = tf.keras.layers.Conv2D(filters = 512 , kernel_size= (3,3), strides = (1,1), padding='same', activation='relu')(X)
    
    X = tfl.BatchNormalization()(X)
    X = tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(2, 2), padding='valid')(X)
    X = tf.keras.layers.Dropout(drop_out)(X)

    X = tf.keras.layers.Conv2D(filters = 1024 , kernel_size= (3,3), strides = (1,1), padding='same', activation='relu')(X)
    X = tf.keras.layers.Conv2D(filters = 1024 , kernel_size= (3,3), strides = (1,1), padding='same', activation='relu')(X)
    X = tf.keras.layers.Conv2D(filters = 1024 , kernel_size= (3,3), strides = (1,1), padding='same', activation='relu')(X)
    
    X = tfl.BatchNormalization()(X)
    X = tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(2, 2), padding='valid')(X)
    X = tf.keras.layers.Dropout(drop_out)(X)

    X = tf.keras.layers.Flatten()(X)


    X = tf.keras.layers.Dense(units=256, activation='relu')(X)
    X = tfl.BatchNormalization(axis = 1)(X)
    X = tf.keras.layers.Dropout(0.5)(X)

    X = tf.keras.layers.Dense(units=200, activation='softmax')(X)
    
    model = tf.keras.Model(inputs=input_img, outputs=X)
    return model

conv_model = convolutional_model((64, 64, 3))
print(conv_model.summary())

conv_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

history = conv_model.fit(train_dataset, validation_data=test_dataset, batch_size=256, epochs=100,workers=-1, callbacks=[tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        min_delta=0.01,
        patience=10,
        verbose=0,
        mode='auto',
        baseline=None,
        restore_best_weights=False
    )])